In [1]:
import os
import time
import random
import argparse
import numpy as np
from tqdm import tqdm
from mixup import mixup_data, mixup_criterion
from utilities import AverageTracker, get_optimizer, showLR, CosineScheduler
from pytorch_nn import Lipreading1
from datasetloadingpy import dataloaders

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
def evaluate(model, dset_loader, criterion):

    model.eval()

    running_loss = 0.
    running_corrects = 0.

    with torch.no_grad():
        for batch_idx, (input, lengths, labels) in enumerate(tqdm(dset_loader)):
            logits = model(input.unsqueeze(1).cuda(), lengths=lengths)
            _, preds = torch.max(F.softmax(logits, dim=1).data, dim=1)
            running_corrects += preds.eq(labels.cuda().view_as(preds)).sum().item()

            loss = criterion(logits, labels.cuda())
            running_loss += loss.item() * input.size(0)

    print('{} in total\tCR: {}'.format( len(dset_loader.dataset), running_corrects/len(dset_loader.dataset)))
    return running_corrects/len(dset_loader.dataset), running_loss/len(dset_loader.dataset)

In [3]:
def train_loop(model, dataloader, criterion, epoch, optimizer):

    data_time = AverageTracker()
    batch_time = AverageTracker()

    mixup_alpha = 0.4

    print("Current Epoch: " + str(epoch))

    model.train()
    running_loss = 0.
    running_corrects = 0.
    running_all = 0.

    end = time.time()
    for batch_idx, (input, lengths, labels) in enumerate(dataloader):

        input, labels_a, labels_b, lam = mixup_data(input, labels, mixup_alpha)
        labels_a, labels_b = labels_a.cuda(), labels_b.cuda()

        optimizer.zero_grad()

        logits = model(input.unsqueeze(1).cuda(), lengths=lengths)

        loss_func = mixup_criterion(labels_a, labels_b, lam)
        loss = loss_func(criterion, logits)

        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        # -- compute running performance
        _, predicted = torch.max(F.softmax(logits, dim=1).data, dim=1)
        running_loss += loss.item()*input.size(0)
        running_corrects += lam * predicted.eq(labels_a.view_as(predicted)).sum().item() + (1 - lam) * predicted.eq(labels_b.view_as(predicted)).sum().item()
        running_all += input.size(0)
    print("Running Loss: {}, Running Corrects: {}, Running All: {}".format(running_loss,running_corrects,running_all))

    return model

In [4]:
data_path = '/home/taylorpap/Bootcamp/CroppedLRW'
temp_words_list = ['ABSOLUTELY', 'BUDGET', 'EVERYONE', 'HOUSE', 'MILITARY', 'PUBLIC', 'RESULT', 'SIGNIFICANT',
                   'WEATHER']
new_temp_words_list = ['BUDGET']
datasets = dataloaders(data_dir=data_path, label_fp=temp_words_list, batch_size=32, workers=8)


Partition train loaded
Partition test loaded
Partition val loaded


In [5]:
train_features, train_lengths, train_labels = next(iter(datasets['train']))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([32, 29, 88, 88])
Labels batch shape: torch.Size([32])


In [6]:
test_model = Lipreading1(len(temp_words_list))

In [7]:
if torch.cuda.is_available():
    test_model.cuda()

In [8]:
test_model

Lipreading1(
  (frontend3D): Sequential(
    (0): Conv3d(1, 64, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (max_pool1): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
  (trunk): ResNet(
    (0): Sequential(
      (0): ResBlock(
        (convs): Sequential(
          (0): ConvLayer(
            (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (1): ConvLayer(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
          (2): ConvLayer(
            (0)

In [9]:
epochs = 40
lr = 3e-3
optimizer = get_optimizer('sgd', optim_policies=test_model.parameters(),init_lr=lr)
criterion = nn.CrossEntropyLoss()
scheduler = CosineScheduler(lr, epochs)
epoch=0

In [10]:
while epoch < epochs:
    start_epoch = time.time()
    model = train_loop(test_model, datasets['train'], criterion, epoch, optimizer)
    acc_avg_val, loss_avg_val = evaluate(model, datasets['val'], criterion)
    print('{} Epoch:\t{:2}\tLoss val: {:.4f}\tAcc val:{:.4f}, LR: {}'.format('val', epoch, loss_avg_val, acc_avg_val, showLR(optimizer)))
    scheduler.adjust_lr(optimizer, epoch)
    epoch +=1
    epoch_len = time.time() - start_epoch
    print("Epoch len: {} Estimated Remaining: {} Min".format(str(epoch_len), str(((epochs-epoch)*epoch_len)/60)))

acc_avg_test, loss_avg_test = evaluate(model, datasets['test'], criterion)
print('Test time performance of best epoch: {} (loss: {})'.format(acc_avg_test, loss_avg_test))

Current Epoch: 0
Running Loss: 22598.049180030823, Running Corrects: 1004.6044528747678, Running All: 8820.0


100%|██████████| 15/15 [00:02<00:00,  5.38it/s]

450 in total	CR: 0.14444444444444443
val Epoch:	 0	Loss val: 2.3341	Acc val:0.1444, LR: 0.0003
Epoch len: 132.33086013793945 Estimated Remaining: 86.01505908966064 Min
Current Epoch: 1


Running Loss: 20106.91296005249, Running Corrects: 1181.358459505753, Running All: 8820.0


100%|██████████| 15/15 [00:02<00:00,  5.52it/s]

...

Running Loss: 6339.239730834961, Running Corrects: 7284.627528490887, Running All: 8820.0


100%|██████████| 15/15 [00:02<00:00,  5.60it/s]


450 in total	CR: 0.9511111111111111
val Epoch:	39	Loss val: 0.2339	Acc val:0.9511, LR: 1.846748910729351e-05
Epoch len: 126.986811876297 Estimated Remaining: 0.0 Min


100%|██████████| 15/15 [00:02<00:00,  5.83it/s]

450 in total	CR: 0.9577777777777777
Test time performance of best epoch: 0.9577777777777777 (loss: 0.23824371230271127)


In [11]:
#ACC of 95.7777 on 9 Words
torch.save(model.state_dict(), '/home/taylorpap/Bootcamp/lipreadlstmv2.pth')